In [ ]:
#@title ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title データを保存するディレクトリのパスを指定
#@markdown ランタイムに接続しドライブを後、コピーしてくる<br>
#@markdown 後ろの / は追加するので、追記は不要<br>

cwd = "/content/drive/MyDrive/RNAseq/Outputs/\u5B9F\u9A13\u540D" #@param {type:"string"}
cwd = cwd + "/"

# 必要なモジュールをインポートする
import pandas as pd
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns

# カレントディレクトリの変更
import os
os.makedirs(cwd , exist_ok = True)
os.chdir(cwd)
currentdir = os.getcwd()

In [ ]:
#@title 実験名を定義
exp_name = "\u5B9F\u9A13\u540D" #@param {type:"string"}
exp_name = exp_name + "_"

In [ ]:
#@title DEGと平均行を追加したTPMデータのパスを定義
df_tpm_path =  "/content/drive/MyDrive/RNA-seq/Outputs/nanopore_Fg1_4/Fg1_4_DEGavetpm.csv" #@param {type:"string"}
df_tpm = pd.read_csv(df_tpm_path , index_col = 0)
print(df_tpm)

In [ ]:
#@title GOファイルのパスを定義

#@markdown ファイルのパスを入力
file_path = "GO\u3084Pathway\u306A\u3069\u306E\u907A\u4F1D\u5B50\u30EA\u30B9\u30C8\u304C\u8A18\u8F09\u3055\u308C\u305F\u30D5\u30A1\u30A4\u30EB\u306E\u30D1\u30B9\u3092\u5165\u529B" #@param {type:"string"}

#@markdown ファイルの形式を選択
file_type = "csv" #@param ["csv", "tsv"]
if file_type == "csv":
  sep_c = ","
else:
  sep_c = "\t"

df_file = pd.read_csv(file_path , sep = sep_c)

df_file

In [ ]:
#@title GOファイルから遺伝子が乗っている列を抽出

#@markdown 読み込んだファイルのどの列に抽出したい遺伝子が記載してあるか入力
Gene_list_column = "\u4E0A\u3067\u8868\u793A\u3055\u308C\u308B\u8868\u3092\u78BA\u8A8D\u3057\u3001\u62BD\u51FA\u3057\u305F\u3044\u907A\u4F1D\u5B50\u540D\u304C\u8A18\u8F09\u3057\u3066\u3042\u308B\u5217\u540D\u3092\u5165\u529B" #@param {type:"string"}

df_list = df_file.loc[: , Gene_list_column]
df_list = df_list.drop_duplicates()
df_list = df_list.sort_values(axis = 0 , ascending = False , ignore_index = True)
df_list

In [ ]:
#@title 抽出元から遺伝子リストに載っているものを抽出する

#@markdown tpmファイルから、遺伝子名が記載してあるカラム名を指定
Gene_column = "product" #@param {type:"string"}

# mergeのinnerメソッドで抽出
df_tyuusyutu = pd.merge(df_list ,
              df_tpm ,
              how = "inner" ,
              left_on = Gene_list_column ,
              right_on = Gene_column
              )

df_tyuusyutu = df_tyuusyutu.drop(Gene_list_column , axis = 1)
# df_tyuusyutu = df_tyuusyutu.drop_duplicates(subset = "product" ,ignore_index = True)

df_tyuusyutu

In [ ]:
#@title 抽出した遺伝子のDEG_log2値をグラフ化
plt.figure(facecolor = "white" , figsize=(len(df_tyuusyutu) / 16 , len(df_tyuusyutu) / 4))
sns.barplot(x = "DEG_log2" , y = "product" , data = df_tyuusyutu)

In [ ]:
#@title N数の定義と必要データを纏めたdfの作成
n_num = 3 #@param {type:"integer"}
df = df_tyuusyutu.copy()

df

In [ ]:
#@title 平均値にエラーバー(標準誤差)をつけたグラフを描画
df_e = df.set_index("product")
df_e.drop(["DEG" , "DEG_log2"] , axis = 1 , inplace = True)

print(df_e)

# for i in range(len(df_e)):
#   if df_e.iloc[i , :].sum(axis = 0) == 0:
#     df_e.drop(df_e.index[i] , axis = 0 , inplace = True)

x = list(df_e.index)
print(x)

df_ave = df_e.iloc[: , [n_num , 2 * n_num + 1]]
df_e.drop(list(df_ave) , axis = 1 , inplace = True)
a = df_e.to_numpy().reshape(len(x) * 2 , n_num)

index_name = []
for gene in x:
  index_name = index_name + [gene + "_Cont" , gene + "_Test"]

y_mean = []
for i in range(len(df_ave)):
  y_mean = y_mean + [df_ave.iloc[i , 0] , df_ave.iloc[i , 1]]

df_e2 = pd.DataFrame(a , index = index_name ,columns = np.arange(n_num) +1)
y_sd = df_e2.std(1) / np.sqrt(n_num)
error_bar_options = {"lw":1 , "capthick":1 , "capsize": 20}

fig = plt.figure(facecolor = "white" , figsize=(len(x) / 4, len(x) / 16))
ax = fig.add_subplot()
ax.bar(index_name , y_mean , yerr = y_sd , error_kw = error_bar_options)

In [ ]:
#@title 平均を使わず、各データの値を比較
a = df_e.to_numpy().reshape(len(x) * 2 * n_num , 1)
b = list(df_e)

index_name = []
hue_name = []
for i in x:
  hue_name = hue_name + b
  for j in range(n_num * 2):
    index_name = index_name + [i]

fig = plt.figure(facecolor = "white" , figsize=(len(x) / 4 , len(x) / 16))
ax = fig.add_subplot()
a = a.flatten()
sns.barplot(x = index_name , y = a , hue = hue_name)